In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [17]:
train = pd.read_csv('open/train.csv')
test = pd.read_csv('open/test.csv')

In [18]:
print(train[:5])
print('=============================')
print(test[:5])

           ID         first_party                    second_party  \
0  TRAIN_0000   Phil A. St. Amant              Herman A. Thompson   
1  TRAIN_0001      Stephen Duncan                  Lawrence Owens   
2  TRAIN_0002   Billy Joe Magwood  Tony Patterson, Warden, et al.   
3  TRAIN_0003          Linkletter                          Walker   
4  TRAIN_0004  William Earl Fikes                         Alabama   

                                               facts  first_party_winner  
0  On June 27, 1962, Phil St. Amant, a candidate ...                   1  
1  Ramon Nelson was riding his bike when he suffe...                   0  
2  An Alabama state court convicted Billy Joe Mag...                   1  
3  Victor Linkletter was convicted in state court...                   0  
4  On April 24, 1953 in Selma, Alabama, an intrud...                   1  
          ID                                        first_party  \
0  TEST_0000                                            Salerno   
1

In [23]:
print(train[:3])

           ID        first_party                    second_party  \
0  TRAIN_0000  Phil A. St. Amant              Herman A. Thompson   
1  TRAIN_0001     Stephen Duncan                  Lawrence Owens   
2  TRAIN_0002  Billy Joe Magwood  Tony Patterson, Warden, et al.   

                                               facts  first_party_winner  
0  On June 27, 1962, Phil St. Amant, a candidate ...                   1  
1  Ramon Nelson was riding his bike when he suffe...                   0  
2  An Alabama state court convicted Billy Joe Mag...                   1  


In [10]:
vectorizer = TfidfVectorizer()

def get_vector(vectorizer, df, train_mode):
    if train_mode:
        X_facts = vectorizer.fit_transform(df['facts'])
    else:
        X_facts = vectorizer.transform(df['facts'])
    X_party1 = vectorizer.transform(df['first_party'])
    X_party2 = vectorizer.transform(df['second_party'])
    
    X = np.concatenate([X_party1.todense(), X_party2.todense(), X_facts.todense()], axis=1)
    return X

In [11]:
print(train['facts'][:5])

0    On June 27, 1962, Phil St. Amant, a candidate ...
1    Ramon Nelson was riding his bike when he suffe...
2    An Alabama state court convicted Billy Joe Mag...
3    Victor Linkletter was convicted in state court...
4    On April 24, 1953 in Selma, Alabama, an intrud...
Name: facts, dtype: object


In [13]:
X_train = get_vector(vectorizer, train, True)
print(X_train[:5].shape)
Y_train = train["first_party_winner"]
X_test = get_vector(vectorizer, test, False)

(5, 52377)


In [6]:
model = LogisticRegression()
model.fit(X_train, Y_train)

/home/moonstar/anaconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


LogisticRegression()

In [14]:
submit = pd.read_csv('open/sample_submission.csv')
print(submit[:5])

          ID  first_party_winner
0  TEST_0000                   0
1  TEST_0001                   0
2  TEST_0002                   0
3  TEST_0003                   0
4  TEST_0004                   0


In [8]:
pred = model.predict(X_test)

/home/moonstar/anaconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [9]:
submit['first_party_winner'] = pred
submit.to_csv('./baseline_submit.csv', index=False)
print('Done')

Done
